https://cloud.google.com/blog/products/gcp/new-in-tensorflow-14-converting-a-keras-model-to-a-tensorflow-estimator
https://github.com/tensorflow/workshops/blob/master/extras/keras-to-estimator/keras-to-estimator.ipynb

## Stack over flow user post classification
### Kerasto estimator

## Load dependencies

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [4]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras

from os.path import expanduser

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.11.0


In [5]:
# https://storage.googleapis.com/tensorflow-workshop-examples/stack-overflow-data.csv

home = expanduser("~")
print(home)

data = pd.read_csv(home + "/.keras/stack-overflow-data.csv")

/root


In [6]:
data.head()

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python


In [7]:
# Confirm that we have a balanced dataset
# Note: data was randomly shuffled in our BigQuery query
data['tags'].value_counts()

objective-c      2000
ruby-on-rails    2000
jquery           2000
angularjs        2000
javascript       2000
c                2000
java             2000
mysql            2000
sql              2000
iphone           2000
python           2000
.net             2000
asp.net          2000
c++              2000
c#               2000
html             2000
android          2000
ios              2000
php              2000
css              2000
Name: tags, dtype: int64

In [8]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 32000
Test size: 8000


In [9]:
train_posts = data['post'][:train_size]
train_tags = data['tags'][:train_size]

test_posts = data['post'][train_size:]
test_tags = data['tags'][train_size:]

In [10]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=False)

In [11]:
tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts).astype(np.float32) # model expects float32
x_test = tokenize.texts_to_matrix(test_posts).astype(np.float32)

In [12]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [13]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes).astype(np.float32)
y_test = keras.utils.to_categorical(y_test, num_classes).astype(np.float32)

In [14]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (32000, 1000)
x_test shape: (8000, 1000)
y_train shape: (32000, 20)
y_test shape: (8000, 20)


In [15]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 2

In [16]:
# Build the model
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, input_shape=(max_words,), name="posts"))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes))
model.add(keras.layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
# Convert Keras model to estimator
estimator_model = keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f18ada22d50>, '_model_dir': '/tmp/tmpCxB_WY', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


In [18]:
# Define input fn
def input_function(features, labels=None, shuffle=False):
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"posts_input": features}, # See the accompanying blog post for explanation on naming
        y=labels,
        shuffle=shuffle
    )
    return input_fn

In [19]:
estimator_model.train(input_fn=input_function(x_train, y_train, shuffle=True))

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpCxB_WY/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpCxB_WY/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: training/Adam/Variable_10; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: training/Adam/Variable_7; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: posts/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: Adam/decay; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: trainin

In [20]:
# Evaluate the accuracy of our trained model
score = estimator_model.evaluate(input_function(x_test, labels=y_test))
print(score)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-05-15:48:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpCxB_WY/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-05-15:48:20
INFO:tensorflow:Saving dict for global step 0: accuracy = 0.048487104, global_step = 0, loss = 3.0422518
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 0: /tmp/tmpCxB_WY/model.ckpt-0
{'loss': 3.0422518, 'global_step': 0, 'accuracy': 0.048487104}


In [21]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_ 

# We'll make predictions for the first five examples
examples = x_test[:5]
predictions = list(estimator_model.predict(input_function(examples)))

# Print out the true and expected label
for i in range(len(examples)):
    prediction_array = list(predictions[i].values())[0]
    predicted_label = text_labels[np.argmax(prediction_array)]
    print('Actual label:' + test_tags.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpCxB_WY/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Actual label:jquery
Predicted label: ios

Actual label:.net
Predicted label: objective-c

Actual label:android
Predicted label: objective-c

Actual label:c++
Predicted label: jquery

Actual label:.net
Predicted label: iphone

